In [1]:
# Se cargan las librerías necesarias para utilizar la red convolucional 
import tensorflow
import keras 
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow.keras import utils
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers import Convolution2D, MaxPooling2D
# from keras.layers import Conv2D
from keras.layers.advanced_activations import LeakyReLU
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# Se definen los parámetros de entrenamiento de la red
import os
path_entreno_2 = '/home/cristian/Escritorio/Reconocedor facial/Datos_desarrollo/Entrenamiento_2/'

epochs = 5
altura, ancho = 250, 250
batch_size = 37
filtro_conv_1 = 32
filtro_conv_2 = 64
tamano_fitro_1 = (5,5)
tamano_filtro_2 = (3,3)
tamano_pool = (2,2)
clases = 2
lr = 0.005 

In [3]:
# Se traen los datos
images = []
labels = []


for path in range(len(os.listdir(path_entreno_2))):
    directorio = path_entreno_2 + os.listdir(path_entreno_2)[path] + '/'
    for image in range(len(os.listdir(directorio))):
        imagen = cv2.imread(path_entreno_2 + os.listdir(path_entreno_2)[path] + '/' + os.listdir(directorio)[image], cv2.IMREAD_GRAYSCALE)
        images.append(imagen)
        labels.append(path)

In [4]:
# Se definen los conjuntos de entrenamiento y test
images, labels = np.array(images), np.array(labels) 
labels = utils.to_categorical(labels, clases)

images_train, images_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = 1, shuffle = True) 
print(images_train.shape, images_test.shape)

(2516, 250, 250) (630, 250, 250)


In [5]:
# Se genera el modelo
modelo = Sequential()
modelo.add(Convolution2D(filters = filtro_conv_1,
    batch_size = batch_size,
    kernel_size = tamano_fitro_1,
    padding = 'same',
    input_shape = (altura, ancho, 1),
    activation = 'relu'))
modelo.add(LeakyReLU(alpha = 0.1))
modelo.add(MaxPooling2D(pool_size = tamano_pool))
modelo.add(Dropout(0.2))
modelo.add(Convolution2D(filters= filtro_conv_2,
                  kernel_size = tamano_filtro_2,
                  activation = 'relu'))
modelo.add(MaxPooling2D(pool_size = tamano_pool))
modelo.add(Dropout(0.2))
modelo.add(Flatten())
modelo.add(Dense(128, activation = 'relu'))
modelo.add(Dense(2, activation = 'softmax'))
modelo.compile(loss = keras.losses.categorical_crossentropy,
               optimizer = optimizers.Adam(lr = lr) ,
               metrics = ['accuracy'])
modelo.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (37, 250, 250, 32)        832       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (37, 250, 250, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (37, 125, 125, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (37, 125, 125, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (37, 123, 123, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (37, 61, 61, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (37, 61, 61, 64)         

In [6]:
# Se redimensionan los datos para que se entrenen
images_train = images_train.reshape(images_train.shape[0],
                                    images_train.shape[1],
                                    images_train.shape[2], 1)

images_test = images_test.reshape(images_test.shape[0],
                                    images_test.shape[1],
                                    images_test.shape[2], 1)



In [7]:
history = modelo.fit(images_train, y_train, 
           epochs = epochs,
           batch_size = batch_size, 
           shuffle = True, 
           validation_data = (images_test, y_test))

Train on 2516 samples, validate on 630 samples
Epoch 1/5
2516/2516 [==============================] - 155s 61ms/step - loss: 569.4489 - accuracy: 0.7421 - val_loss: 0.5632 - val_accuracy: 0.7282
Epoch 2/5
2516/2516 [==============================] - 166s 66ms/step - loss: 0.5548 - accuracy: 0.7703 - val_loss: 0.5357 - val_accuracy: 0.7282
Epoch 3/5
2516/2516 [==============================] - 168s 67ms/step - loss: 0.5466 - accuracy: 0.7703 - val_loss: 0.5382 - val_accuracy: 0.7282
Epoch 4/5
2516/2516 [==============================] - 165s 66ms/step - loss: 0.5428 - accuracy: 0.7703 - val_loss: 0.5398 - val_accuracy: 0.7282
Epoch 5/5
2516/2516 [==============================] - 162s 64ms/step - loss: 0.5475 - accuracy: 0.7703 - val_loss: 0.5392 - val_accuracy: 0.7282


In [11]:
# Se calcula la precisión de la red con los datos de test
test_evaluate = modelo.evaluate(images_test, y_test, verbose = 1, batch_size = batch_size)
test_evaluate[0]
test_evaluate[1]


630/630 [==============================] - 6s 10ms/step


0.7282282114028931